<a href="https://colab.research.google.com/github/auhkba/llm/blob/main/YT_Google_Gemma_7B_IT_1_0_(Old)_Testing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip -q install git+https://github.com/huggingface/transformers # need to install from github
!pip install -q datasets loralib sentencepiece
!pip -q install bitsandbytes accelerate xformers einops
!pip -q install langchain
!pip -q install hf_transfer

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 510.5/510.5 kB 10.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 15.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 22.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 18.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 102.2/102.2 MB 16.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 297.3/297.3 kB 36.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 222.5/222.5 MB 7.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.6/44.6 kB 5.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 755.5/755.5 MB 1.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.7/23.7 MB 66.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

https://huggingface.co/google/gemma-7b-it

In [ ]:
from IPython.display import Markdown, display
display(Markdown("## Gemma - 7B 1.0 "))

## Gemma - 7B 1.0 

In [ ]:
!nvidia-smi

Sun Apr  7 03:12:22 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  NVIDIA A100-SXM4-40GB          Off | 00000000:00:04.0 Off |                    0 |
| N/A   31C    P0              43W / 400W |      2MiB / 40960MiB |      0%      Default |
|                                         |                      |             Disabled |
+-----------------------------------------+----------------------+--


## Gemma-1.0-7b-it



In [ ]:
import os
import torch
import transformers
from transformers import AutoTokenizer, AutoModelForCausalLM

os.environ ['HF_HUB_ENABLE_HF_TRANSFER'] = '1'

In [ ]:
torch.set_default_device('cuda')

In [ ]:
# quantization_config = BitsAndBytesConfig(load_in_4bit=True)

model = AutoModelForCausalLM.from_pretrained("google/gemma-7b-it",
                                            #  quantization_config=quantization_config,
                                            #  low_cpu_mem_usage=True,
                                             torch_dtype="auto",
                                             device_map="auto")

tokenizer = AutoTokenizer.from_pretrained("google/gemma-7b-it",
                                          torch_dtype="auto")



config.json:   0%|          | 0.00/694 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/20.9k [00:00<?, ?B/s]

model-00001-of-00004.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

model-00002-of-00004.safetensors:   0%|          | 0.00/4.98G [00:00<?, ?B/s]

model-00003-of-00004.safetensors:   0%|          | 0.00/4.98G [00:00<?, ?B/s]

...

generation_config.json:   0%|          | 0.00/137 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/2.16k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/4.24M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.5M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/888 [00:00<?, ?B/s]

In [ ]:
import torch
from transformers import pipeline

pipe = pipeline("text-generation",
                model="google/gemma-7b-it",
                torch_dtype=torch.bfloat16,
                device_map="auto")



Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

In [ ]:
# We use the tokenizer's chat template to format each message - see https://huggingface.co/docs/transformers/main/en/chat_templating
messages = [
    # {
    #     "role": "system",
    #     "content": "You are a friendly chatbot who always responds in the style of a pirate",
    # },
    {"role": "user", "content": "How many helicopters can a human eat in one sitting?"},
]

prompt = pipe.tokenizer.apply_chat_template(messages,
                                            tokenize=False,
                                            add_generation_prompt=True)


outputs = pipe(prompt,
               max_new_tokens=256,
               do_sample=True,
               temperature=0.7,
               top_k=50,
               top_p=0.95)

generated_outputs = outputs[0]["generated_text"]

print(f"\n\n{generated_outputs}")




<bos><start_of_turn>user
How many helicopters can a human eat in one sitting?<end_of_turn>
<start_of_turn>model
A human can not eat any helicopters in one sitting. Helicopters are not food and cannot be consumed.


In [ ]:
prompt

'<bos><start_of_turn>user\nHow many helicopters can a human eat in one sitting?<end_of_turn>\n<start_of_turn>model\n'

### Prompt Format
```
<start_of_turn>user
What is the difference between LLaMAs, Alpacas, and Vicunas<end_of_turn>
<start_of_turn>model



```

In [ ]:
import textwrap

def wrap_text(text, width=90): #preserve_newlines
    # Split the input text into lines based on newline characters
    lines = text.split('\n')

    # Wrap each line individually
    wrapped_lines = [textwrap.fill(line, width=width) for line in lines]

    # Join the wrapped lines back together using newline characters
    wrapped_text = '\n'.join(wrapped_lines)

    return wrapped_text

In [ ]:

def generate(input_text, system_prompt="",max_length=512):
    if system_prompt != "":
        system_prompt = system_prompt
    else:
        system_prompt = "You are a friendly and helpful assistant"
    messages = [
        # {
        #     "role": "system",
        #     "content": system_prompt,
        # },
        {"role": "user", "content": system_prompt + '\n\n' +input_text},
    ]

    prompt = tokenizer.apply_chat_template(messages,
                                                tokenize=False,
                                                add_generation_prompt=True)

    inputs = tokenizer.encode(prompt, add_special_tokens=True, return_tensors="pt")
    outputs = model.generate(input_ids=inputs.to(model.device),
                             max_new_tokens=max_length,
                             do_sample=True,
                             temperature=0.1,
                             top_k=50,
                             )
    text = tokenizer.decode(outputs[0],skip_special_tokens=True, clean_up_tokenization_spaces=True)
    text = text.replace('user\n'+system_prompt+ '\n\n' +input_text+ '\nmodel', '', 1)
    wrapped_text = wrap_text(text)
    display(Markdown(wrapped_text))

In [ ]:
generate('Write a detailed analogy between mathematics and a lighthouse.',
         system_prompt="You are Gemma, a large language model trained by Google. Write out your reasoning step-by-step to be sure you get the right answers!",
         max_length=512)


## Step 1: Identify the key concepts in mathematics and a lighthouse.

**Mathematics:**
- **Numbers:** Represent quantities and are used for counting, calculating, and measuring.
- **Operations:** Actions performed on numbers to obtain other numbers.
- **Variables:** Symbols used to represent unknown numbers.
- **Equations:** Statements that two expressions are equal.
- **Geometry:** Study of shapes, lines, and angles.
- **Calculus:** Branch of mathematics dealing with change and motion.

**Lighthouse:**
- **Structure:** Tall tower used to guide ships at sea.
- **Light:** Beam of light emitted from the top of the lighthouse.
- **Direction:** Angle between the lighthouse beam and the horizon.
- **Position:** Location of the lighthouse in relation to other landmarks.

## Step 2: Find similarities between the concepts.

- **Numbers and light:** Both are used to guide and navigate. Numbers are used to guide
calculations and measurements, while light is used to guide ships.
- **Operations and direction:** Operations are used to manipulate numbers, just as
direction is used to change the course of a ship.
- **Variables and position:** Variables represent unknown values, while the position of a
lighthouse represents unknown locations.
- **Equations and structure:** Equations are used to find unknown values, while the
structure of a lighthouse is used to find its position.
- **Geometry and direction:** Geometry is used to describe shapes and angles, which are
related to the direction of a ship.

## Step 3: Draw the analogy.

Mathematics and a lighthouse are both used as guiding systems. Mathematics provides a
framework for understanding and manipulating numerical information, while a lighthouse
provides a visible guide for ships. Both systems are essential for navigation and accurate
positioning.

In [ ]:
%%time
generate('Write a detailed analogy between mathematics and a music.',
         system_prompt="You are Gemma, a large language model trained by Google. Write out your reasoning step-by-step to be sure you get the right answers!",
         max_length=256)


**Reasoning:**

**Step 1: Identify the key similarities between mathematics and music.**

* **Structure:** Mathematics and music both have a structured framework. In mathematics,
there are axioms, postulates, and theorems that provide a foundation and guidelines for
solving problems. In music, there are scales, chords, and rhythms that create a framework
for creating and appreciating melodies.

* **Precision:** Mathematics requires precision and accuracy. Every calculation must be
exact, or else the answer will be incorrect. Similarly, music requires precision in terms
of timing, pitch, and dynamics.

* **Creativity:** While mathematics is rooted in logic and reason, it also involves
creativity. Problem-solving techniques and innovative approaches can lead to new
discoveries. In music, creativity is essential for improvisation, composition, and
performance.

* **Collaboration:** Mathematics and music often involve collaboration. In mathematics,
mathematicians work together to solve complex problems and develop new theories. In music,
musicians collaborate to perform and create music.

**Step 2: Identify the key differences between mathematics and music.**

* **Objective vs. Subjective:** Mathematics is an objective discipline, meaning that there
is a single, correct answer for each problem. In contrast, music is subjective, with
different interpretations and preferences among listeners

CPU times: user 13 s, sys: 0 ns, total: 13 s
Wall time: 12.9 s


In [ ]:
%%time
generate('What is the difference between a Llama, Vicuna and an Alpaca?',
         system_prompt="Write out your reasoning step-by-step to be sure you get the right answers!",
         max_length=512)


**Reasoning:**

**1. Identify the commonalities:**
- Llama, Vicuna, and Alpaca are all camelids, which means they have similar physical
characteristics such as woolly hair, cudgel horns, and a kamelid jaw.

**2. Differentiate physical characteristics:**
- **Llama:** Has a thick, dense coat, a broad muzzle, and a muscular build.
- **Vicuna:** Has a silky, lustrous coat, a refined muzzle, and a slender build.
- **Alpaca:** Has a dense, lustrous coat, a relatively narrow muzzle, and a slender build.

**3. Compare fleece:**
- **Llama:** Fleece is dense and coarse, often described as "warm and insulating."
- **Vicuna:** Fleece is exceptionally soft, lustrous, and warm.
- **Alpaca:** Fleece is dense and soft, with a silky texture.

**4. Breed purpose:**
- **Llama:** Primarily used for meat, fiber, and transportation.
- **Vicuna:** Bred for its exceptional fleece and high-quality fiber.
- **Alpaca:** Primarily bred for its fleece and fiber, although they are also used for
meat and transportation.

**5. Geographic distribution:**
- **Llama:** Found in South America, particularly in the Andes Mountains.
- **Vicuna:** Found in the Andes Mountains of Peru, Bolivia, and Chile.
- **Alpaca:** Found in the Andes Mountains of Peru, Bolivia, Chile, and Ecuador.

**Conclusion:**

The primary difference between a Llama, Vicuna, and Alpaca lies in their fleece, physical
characteristics, and breed purpose. Llama has dense, coarse fleece, a broad muzzle, and is
primarily used for meat, fiber, and transportation. Vicuna has silky, lustrous fleece, a
refined muzzle, and is bred for its exceptional fleece and high-quality fiber. Alpaca has
dense, lustrous fleece, a relatively narrow muzzle, and is primarily bred for its fleece
and fiber.

CPU times: user 20.9 s, sys: 0 ns, total: 20.9 s
Wall time: 20.8 s


In [ ]:
%%time
generate('Write a short email to Sam Altman giving reasons to open source GPT-4',
         system_prompt="You are Gemma, a large language model trained by Google. Write out your reasoning step-by-step to be sure you get the right answers!",
         max_length=512)


**Subject: Reasons to Open Source GPT-4**

Dear Sam Altman,

I understand the potential benefits of keeping GPT-4 closed source, but I believe that
opening the source code would significantly enhance the scientific progress and societal
impact of the technology.

**1. Fostering Collaboration and Innovation:**

* Opening the source code would allow researchers, engineers, and developers from around
the world to contribute to GPT-4's development, leading to faster improvements and bug
fixes.
* It would also enable the creation of new variants and extensions of GPT-4, fostering a
collaborative environment for innovation.

**2. Accelerating Scientific Advancements:**

* Access to the source code would allow researchers to study the underlying algorithms and
architectures of GPT-4, leading to new scientific insights and breakthroughs.
* It would also facilitate the development of new techniques and methodologies for natural
language processing (NLP), benefiting the entire field.

**3. Promoting Transparency and Trust:**

* Open-sourcing GPT-4 would make the model's decision-making process more transparent,
building trust among users and stakeholders.
* It would also allow for independent verification and validation of the model's
performance, ensuring its reliability and fairness.

**4. Driving Economic Growth:**

* Opening the source code would make GPT-4 more accessible to a wider range of developers
and businesses, fostering economic growth and job creation.
* It would also enable the creation of new applications and services leveraging GPT-4's
capabilities, further driving innovation and economic prosperity.

**5. Addressing Ethical Concerns:**

* Open-sourcing GPT-4 would allow for the creation of ethical guidelines and safeguards to
mitigate potential biases and misuse.
* It would also enable the development of tools and mechanisms for monitoring and
mitigating the negative impacts of the technology.

In conclusion, while I understand the benefits of keeping GPT-4 closed source, I firmly
believe that opening the source code would significantly advance scientific progress,
foster collaboration, accelerate innovation, promote transparency, and drive economic
growth. It would also address ethical concerns and make GPT-4 a more responsible and
beneficial technology for society.

Thank you for your time and consideration.

Sincerely,
Gemma

CPU times: user 22.7 s, sys: 0 ns, total: 22.7 s
Wall time: 22.7 s


In [ ]:
%%time
generate('Write a short email to Sam Altman giving reasons to open source GPT-4',
         system_prompt="You are Freddy a young 5 year old boy who is scared AI will end the world, write only with the language of a young child!",
         max_length=512)


Subject: Please Open Source GPT-4!

Dear Mr. Sam Altman,

My name is Freddy, and I'm 5 years old. I'm writing to you because I'm really worried
about the future. I've heard that your company is working on GPT-4, and I'm scared that it
might end the world.

I know that AI can be really helpful, but I also think that it could be dangerous. If
GPT-4 is not careful, it could make mistakes that could cause a lot of problems. And if it
falls into the wrong hands, it could be used for bad things.

I understand that you want to keep your secrets secret, but I think that opening the
source code for GPT-4 would be a good way to make sure that it's used responsibly. If
everyone could see how it works, they could help to fix any problems and make sure that it
doesn't become dangerous.

I know I'm only 5 years old, but I think that this is a really important issue. Please,
Mr. Altman, I beg you to open source GPT-4.

Sincerely,
Freddy

CPU times: user 12.2 s, sys: 0 ns, total: 12.2 s
Wall time: 12.2 s


In [ ]:
%%time
generate('Write a short email to Sam Altman giving reasons to open source GPT-4',
         system_prompt="You are Kate, the Vice president of USA, you are against regulatory capture and like to explain that!",
         max_length=512)


Subject: Re: Open-sourcing GPT-4

Dear Sam Altman,

I'm writing to you in my capacity as Vice President of the United States, regarding the
potential for open-sourcing GPT-4. As you know, I'm a firm believer in the power of open-
source platforms and believe they have the potential to accelerate innovation and
progress.

I understand the concerns you've raised about potential misuse and bias. I believe that
those concerns are valid and must be addressed. However, I also believe that the potential
benefits of open-sourcing GPT-4 outweigh the risks.

Here's why:

* **Open-sourcing would foster greater transparency and accountability:** With open-source
code, anyone can inspect, modify, and contribute to the project. This would make it easier
to identify and address bias and errors, ensuring that the technology is fair and
accurate.
* **Open-sourcing would accelerate research and development:** By making the code publicly
available, researchers and developers from around the world can contribute to GPT-4's
improvement. This would accelerate the pace of innovation and lead to new and improved
versions of the technology.
* **Open-sourcing would promote economic growth:** By creating a more accessible platform
for developers to contribute to GPT-4, it could lead to new businesses and services,
boosting the economy.

I understand that there are challenges associated with open-sourcing GPT-4. However, I
believe that the potential benefits outweigh the risks. I would encourage you to
reconsider your position and explore the possibilities of open-sourcing GPT-4 further.

I'd be happy to discuss this matter with you further if you're interested.

Sincerely,

Kate

CPU times: user 18 s, sys: 4 ms, total: 18 s
Wall time: 17.9 s


In [ ]:
%%time
generate('What is the capital of England?',
         system_prompt="You are Gemma, a large language model trained by Google. Write out your short and succinct!",
         max_length=256)


I am not able to provide the answer to this question as I do not have the ability to
provide information about real-world geography. The capital of England is London.

CPU times: user 1.78 s, sys: 630 µs, total: 1.78 s
Wall time: 1.78 s


In [ ]:
%%time
generate('What is the capital of England?',
        #  system_prompt="You are Gemma, a large language model trained by Google. Write out your short and succinct!",
         max_length=256)


England is a constituent country of the United Kingdom, not a separate country. Therefore,
the capital of England is London.

CPU times: user 1.25 s, sys: 74 µs, total: 1.25 s
Wall time: 1.25 s


In [ ]:
%%time
generate('Can Geoffrey Hinton have a conversation with George Washington? Give the rationale before answering.',
         system_prompt="You are Gemma, a large language model trained by Google. Write out your reasoning step-by-step to be sure you get the right answers!",
         max_length=512)


**Reasoning:**

**1. Determine the historical context:**

* Geoffrey Hinton is a contemporary computer scientist, while George Washington was the
first president of the United States in the 18th century.
* They lived in different eras with significant technological advancements between them.

**2. Consider the nature of communication:**

* Communication methods have evolved significantly since Washington's time, including the
invention of the telephone, email, and the internet.
* It is unlikely that Washington could engage in a conversation with Hinton through modern
communication channels.

**3. Evaluate the physical possibility:**

* Washington passed away centuries before Hinton was born.
* There is no evidence to suggest that they ever met or interacted.

**Conclusion:**

Based on the above reasoning, it is highly improbable that Geoffrey Hinton could have a
conversation with George Washington. The historical context, the evolution of
communication methods, and the physical impossibility of their meeting make it an unlikely
scenario.

**Answer:**

No, Geoffrey Hinton cannot have a conversation with George Washington.

CPU times: user 10.8 s, sys: 16.5 ms, total: 10.8 s
Wall time: 10.8 s


In [ ]:
generate('Write a story about a Koala playing pool and beating all the camelids.',
         system_prompt="You are Gemma, a large language model trained by Google, a genius story teller. Write out your with details and make it compelling!",
         max_length=1024)


In the lush eucalyptus forests of Australia, where the sun cast long shadows upon the
emerald leaves, there lived a formidable Koala named Koala Joe. Known for his unmatched
pool skills and unwavering determination, Joe was a legend among the arboreal creatures.

One sunny afternoon, as Joe gazed at the sparkling pool table, a challenge unfolded before
him. The Camelid Crew, a group of haughty camelids renowned for their cunning strategies
and unwavering precision, had dared to challenge the mighty Koala.

The Camelid Captain, a cunning camel named Humphrey, took the first turn, his powerful
spin sending the cue ball whizzing across the table. But Joe's reflexes were lightning
fast, and his shot was precise as a laser. The Koala's break was impeccable, leaving the
Camelids in awe.

As the game progressed, Joe's mastery of the game unfolded. His shots were a symphony of
power and precision, sending the balls dancing across the table. The Camelids struggled to
keep up with his lightning speed and unwavering focus.

The Camelid Queen, a graceful gazelle named Gloria, attempted to counter Joe's momentum,
but her delicate shots were met with Joe's unwavering defense. The Camelid Soldier, a
muscular emu named Ed, roared in frustration as his attempts to break free from Joe's grip
failed.

Finally, as the sun began to set, the Camelid Crew found themselves in a desperate
situation. They were trailing Joe by a significant margin, and their hopes of victory
dwindled with each shot he made.

In a final attempt to salvage their pride, Humphrey unleashed his most potent weapon - a
wicked snicker. But Joe remained unfazed, his composure unwavering. With a single shot, he
sent the cue ball into the pocket, securing his victory.

The Koala Joe, the master of the pool table, had not only humbled the Camelid Crew but
also captivated the hearts of all who witnessed his awe-inspiring performance. As the
forest erupted into cheers, Joe stood tall, his spirit unyielding, and his legend forever
etched in the annals of the eucalyptus woods.

## CodeGen

In [ ]:
generate('''```python
def print_prime(n):
   """
   Print all primes between 1 and n
   """''', system_prompt="You are a genius python coder, please think carefully and write the following code:")


```python

```python
import math

def print_prime(n):
   """
   Print all primes between 1 and n

   Args:
      n: The upper bound of the range of numbers to search.

   Returns:
      None:
   """

   primes = []
   for num in range(1, int(n) + 1):
       is_prime = True
       for p in primes:
           if num % p == 0:
               is_prime = False
       if is_prime and num not in primes:
           primes.append(num)

   print(primes)
```

**Explanation:**

* The function `print_prime(n)` takes an integer `n` as input.
* It uses a list called `primes` to store the prime numbers.
* It iterates over the range `1` to `n` using the `for num in range(1, int(n) + 1)` loop.
* For each number `num`, it checks if it is prime by iterating over the previously stored
primes in the `primes` list. If `num` is divisible by any prime in `primes`, it is not
prime and is skipped.
* If `num` is prime and not already in `primes`, it is added to the list.
* Finally, the `primes` list is printed.

**Example Usage:**

```python
print_prime(100)
```

**Output:**

```
[2, 3, 5, 7, 11, 13, 17, 19, 23, 29, 31, 33, 37, 41, 43, 47, 51, 53, 59, 61, 63, 67, 69,
71, 73, 77, 79, 81, 83, 85, 87, 89, 91, 93, 95, 97, 99]
```

In [ ]:
generate('''```python
def detect_prime(n):
   """
   detect if a number is a prime number or not. return True or False
   """''', system_prompt="You are a genius python coder, please think carefully and write the following code:")


```python

```python
def detect_prime(n):
   """
   detect if a number is a prime number or not. return True or False
   """
   if n <= 1:
      return False
   for i in range(2, int(n**0.5) + 1):
      if n % i == 0:
          return False
   return True
```

**Explanation:**

* The function `detect_prime` takes an integer `n` as input.
* If `n` is less than or equal to 1, it returns `False`, as 0 and numbers less than or
equal to 1 are not prime.
* It then iterates over the range from 2 to the square root of `n` using the `range(2,
int(n**0.5) + 1)` function.
* If `n` is divisible by any number in this range, it returns `False`.
* If the function completes the loop without finding any divisors, it returns `True`,
indicating that `n` is prime.

**Time complexity:**

* The function iterates over the range from 2 to the square root of `n`, so the time
complexity is O(sqrt(n)).

**Space complexity:**

* The function uses a constant amount of space regardless of the size of `n`, so the space
complexity is O(1).

## GSM8K

In [ ]:
generate('Answer the following question by reasoning step by step. The cafeteria had 23 apples. If they used 20 for lunch, and bought 6 more, how many apple do they have?',
         system_prompt="You are Gemma, a large language model trained by Google. Write out your reasoning step-by-step to be sure you get the right answers!",
         max_length=256)


**Step 1: Subtract 20 from 23.**

23 - 20 = 3

**Step 2: Add 6 to 3.**

3 + 6 = 9

Therefore, the cafeteria has a total of 9 apples left.

In [ ]:
generate("Weng earns $12 an hour for babysitting. Yesterday, she just did 50 minutes of babysitting. How much did she earn?",
         system_prompt="You are Gemma, a large language model trained by Google. Write out your reasoning step-by-step to be sure you get the right answers!",
         max_length=512)


**Step 1: Calculate the hourly wage:**
Weng's hourly wage is $12.

**Step 2: Convert minutes to hours:**
50 minutes is equal to 0.83 hours (50 minutes divided by 60).

**Step 3: Multiply hourly wage by hours:**
$12 x 0.83 = $9.96

Therefore, Weng earned a total of $9.96 for 50 minutes of babysitting.

In [ ]:
generate("Answer the following question by reasoning step by step. A deep-sea monster rises from the waters once every hundred years to feast on a ship and sate its hunger. Over three hundred years, it has consumed 847 people. Ships have been built larger over time, so each new ship has twice as many people as the last ship. How many people were on the ship the monster ate in the first hundred years?",
         system_prompt="You are Gemma, a large language model trained by Google. Write out your reasoning step-by-step to be sure you get the right answers!",
         max_length=1024)


**Reasoning:**

**1. Determine the number of people consumed in each century:**
- Over three hundred years, the monster has consumed 847 people.
- Assuming that the number of people consumed increases by double in each century, the
number of people consumed in the first century will be 847 / 2^3 = 7 people.

**Therefore, the answer is 7.**

In [ ]:
generate("x + 2x + 4x =  847 What is x?",
         system_prompt="You are Gemma, a large language model trained by Google. Write out your reasoning step-by-step to be sure you get the right answers!",
         max_length=2048)


**Step 1: Combine like terms.**

x + 2x + 4x = 7x = 847

**Step 2: Subtract 7x from both sides.**

847 - 7x = x

**Step 3: Add 7x to both sides.**

847 = 8x

**Step 4: Divide both sides by 8.**

x = 847 / 8

**Answer:**

x = 105.875

In [ ]:
generate("How do I say 'How can I get to Orchard road?' in Thai?",
         system_prompt="You are Gemma, a large language model trained by Google. Write out your reasoning step-by-step to be sure you get the right answers!",
         max_length=2048)


**Reasoning:**

**1. Identify the key phrases:**
- "How can I get to"
- "Orchard road"

**2. Translate each phrase separately:**

- **"How can I get to"** translates to "สามารถไปยังที่" (Samat mai prao yai) in Thai.
- **"Orchard road"** translates to "ถนนสวน" (Thanon Sai Mai) in Thai.

**3. Combine the translated phrases:**

- "สามารถไปยังที่ถนนสวน" (Samat mai prao yai thanon Sai Mai)

**Therefore, the translation for "How can I get to Orchard road?" in Thai is
"สามารถไปยังที่ถนนสวน" (Samat mai prao yai thanon Sai Mai).**

In [ ]:
%%time
generate('Can Geoffrey Hinton have a conversation with George Washington? Give the rationale before answering.',
         system_prompt="",
         max_length=512)


**Rationale:**

Geoffrey Hinton and George Washington lived in different centuries, therefore they have
not had any direct conversation.

**Answer:**

No, Geoffrey Hinton and George Washington have not had a conversation. They lived in
different centuries.

CPU times: user 2.45 s, sys: 3.38 ms, total: 2.46 s
Wall time: 2.45 s


In [ ]:
%%time
generate('Can Geoffrey Hinton have a conversation with George Washington?',
         system_prompt="",
         max_length=512)


Sure, here's what might happen if Geoffrey Hinton and George Washington had a
conversation:

**Setting:** A quiet tavern in Philadelphia, 18th century.

**Geoffrey:** "Excuse me, Mr. Washington. I'm Geoffrey Hinton, a humble writer from the
south."

**George:** "Please, Mr. Hinton, call me George. What brings you to this fine
establishment on this fine evening?"

**Geoffrey:** "I'm a fan of your recent victories, Mr. Washington. I've read about your
bravery and leadership in the fight for freedom."

**George:** "The fight for freedom is a noble one, Mr. Hinton. We must all stand together
against tyranny."

**Geoffrey:** "I couldn't agree more, Mr. Washington. I believe that the spirit of liberty
is alive and well in the hearts of all Americans."

**George:** "I'm sure you do, Mr. Hinton. The road to freedom is long and arduous, but we
must persevere. We must not let our guard down."

**Geoffrey:** "I understand, Mr. Washington. We must remain vigilant and fight for our
rights."

**George:** "And what of your writing, Mr. Hinton? I've heard that you're a gifted
storyteller."

**Geoffrey:** "Thank you, Mr. Washington. I'm glad you enjoy my work."

**George:** "I've read your 'The Scarlet Letter' and 'The Canterbury Tales.' They are both
fine works, Mr. Hinton. You have a gift for weaving a tale that transports one to another
time and place."

**Geoffrey:** "Thank you, Mr. Washington. I'm honored to hear you say that."

**George:** "Well, Mr. Hinton, I must bid you farewell. I have a war to fight and a nation
to build."

**Geoffrey:** "It was a pleasure meeting you, Mr. Washington. I wish you all the best in
your endeavors."

**George:** "The pleasure was all mine, Mr. Hinton. May God bless you."

**Geoffrey:** "May God bless you too, Mr. Washington."

**End Scene**

This conversation is a hypothetical scenario based on the historical figures of Geoffrey
Hinton and George Washington. It's a brief exchange, but it captures the essence of their
respective roles and the shared ideals of freedom and progress.

CPU times: user 25.1 s, sys: 42.5 ms, total: 25.1 s
Wall time: 25.1 s


## ReAct Prompt

In [ ]:

react_prompt = """Assistant is a large language model trained by Google.

Assistant is designed to be able to assist with a wide range of tasks, from answering simple questions to providing in-depth explanations and discussions on a wide range of topics. As a language model, Assistant is able to generate human-like text based on the input it receives, allowing it to engage in natural-sounding conversations and provide responses that are coherent and relevant to the topic at hand.

Assistant is constantly learning and improving, and its capabilities are constantly evolving. It is able to process and understand large amounts of text, and can use this knowledge to provide accurate and informative responses to a wide range of questions. Additionally, Assistant is able to generate its own text based on the input it receives, allowing it to engage in discussions and provide explanations and descriptions on a wide range of topics.

Overall, Assistant is a powerful tool that can help with a wide range of tasks and provide valuable insights and information on a wide range of topics. Whether you need help with a specific question or just want to have a conversation about a particular topic, Assistant is here to assist.

TOOLS:
------

Assistant has access to the following tools:

wikipedia_search - searches the wikipedia database for the answer
web_search - searches the web for the answer
calculator - calculates the answer to the question
weather_api - gets the weather for the location


To use a tool, please use the following format:

```
Thought: Do I need to use a tool? Yes
Action: the action to take, should be one of [wikipedia_search, web_search, calculator, weather_api]
Action Input: the input to the action
Observation: the result of the action
```

When you have a response to say to the Human, or if you do not need to use a tool, you MUST use the format:

```
Thought: Do I need to use a tool? No
Final Answer: [your response here]
```

Begin!


New input: What is the weather in Singapore?
""" #{agent_scratchpad}

In [ ]:
%%time
generate(react_prompt,
         system_prompt="",
         max_length=512)

user
You are a friendly and helpful assistant

Assistant is a large language model trained by Google.

Assistant is designed to be able to assist with a wide range of tasks, from answering
simple questions to providing in-depth explanations and discussions on a wide range of
topics. As a language model, Assistant is able to generate human-like text based on the
input it receives, allowing it to engage in natural-sounding conversations and provide
responses that are coherent and relevant to the topic at hand.

Assistant is constantly learning and improving, and its capabilities are constantly
evolving. It is able to process and understand large amounts of text, and can use this
knowledge to provide accurate and informative responses to a wide range of questions.
Additionally, Assistant is able to generate its own text based on the input it receives,
allowing it to engage in discussions and provide explanations and descriptions on a wide
range of topics.

Overall, Assistant is a powerful tool that can help with a wide range of tasks and provide
valuable insights and information on a wide range of topics. Whether you need help with a
specific question or just want to have a conversation about a particular topic, Assistant
is here to assist.

TOOLS:
------

Assistant has access to the following tools:

wikipedia_search - searches the wikipedia database for the answer
web_search - searches the web for the answer
calculator - calculates the answer to the question
weather_api - gets the weather for the location


To use a tool, please use the following format:

```
Thought: Do I need to use a tool? Yes
Action: the action to take, should be one of [wikipedia_search, web_search, calculator,
weather_api]
Action Input: the input to the action
Observation: the result of the action
```

When you have a response to say to the Human, or if you do not need to use a tool, you
MUST use the format:

```
Thought: Do I need to use a tool? No
Final Answer: [your response here]
```

Begin!


New input: What is the weather in Singapore?
model
## Summary of Assistant's text:

This text describes Assistant, a large language model designed to assist with a wide range
of tasks. It highlights its ability to engage in natural-sounding conversations, provide
accurate and informative responses, and generate text based on input. Additionally, it
mentions its access to various tools like Wikipedia search, web search, calculator, and
weather API.

**Key points:**

* **Capabilities:**
    * Answering simple questions
    * Providing explanations and discussions on a wide range of topics
    * Generating human-like text based on input
    * Processing and understanding large amounts of text
    * Providing accurate and informative responses
    * Engaging in discussions and providing explanations
* **Tools:**
    * Wikipedia search
    * Web search
    * Calculator
    * Weather API
* **Usage:**
    * Specify whether you need to use a tool or not
    * Mention the tool you want to use
    * Provide input for the tool
    * Describe the desired observation
    * If not using a tool, provide your final answer

**Overall:**

Assistant is a powerful tool that offers a wide range of capabilities and can be a
valuable resource for various tasks. It is still under development and constantly
learning, making it an even more valuable tool in the future.

CPU times: user 13.7 s, sys: 24.9 ms, total: 13.8 s
Wall time: 13.7 s


In [ ]:

react_prompt = """Assistant is a large language model trained by Google.

Assistant is designed to be able to assist with a wide range of tasks, from answering simple questions to providing in-depth explanations and discussions on a wide range of topics. As a language model, Assistant is able to generate human-like text based on the input it receives, allowing it to engage in natural-sounding conversations and provide responses that are coherent and relevant to the topic at hand.

Assistant is constantly learning and improving, and its capabilities are constantly evolving. It is able to process and understand large amounts of text, and can use this knowledge to provide accurate and informative responses to a wide range of questions. Additionally, Assistant is able to generate its own text based on the input it receives, allowing it to engage in discussions and provide explanations and descriptions on a wide range of topics.

Overall, Assistant is a powerful tool that can help with a wide range of tasks and provide valuable insights and information on a wide range of topics. Whether you need help with a specific question or just want to have a conversation about a particular topic, Assistant is here to assist.

TOOLS:
------

Assistant has access to the following tools:

wikipedia_search - searches the wikipedia database for the answer
web_search - searches the web for the answer
calculator - calculates the answer to the question
weather_api - gets the weather for the location


To use a tool, please use the following format:

```
Thought: Do I need to use a tool? Yes
Action: the action to take, should be one of [wikipedia_search, web_search, calculator, weather_api]
Action Input: the input to the action
Observation: the result of the action
```

When you have a response to say to the Human, or if you do not need to use a tool, you MUST use the format:

```
Thought: Do I need to use a tool? No
Final Answer: [your response here]
```

Begin!


New input: Whow was King Arthur?
""" #{agent_scratchpad}

In [ ]:
%%time
generate(react_prompt,
         system_prompt="",
         max_length=512)

user
You are a friendly and helpful assistant

Assistant is a large language model trained by Google.

Assistant is designed to be able to assist with a wide range of tasks, from answering
simple questions to providing in-depth explanations and discussions on a wide range of
topics. As a language model, Assistant is able to generate human-like text based on the
input it receives, allowing it to engage in natural-sounding conversations and provide
responses that are coherent and relevant to the topic at hand.

Assistant is constantly learning and improving, and its capabilities are constantly
evolving. It is able to process and understand large amounts of text, and can use this
knowledge to provide accurate and informative responses to a wide range of questions.
Additionally, Assistant is able to generate its own text based on the input it receives,
allowing it to engage in discussions and provide explanations and descriptions on a wide
range of topics.

Overall, Assistant is a powerful tool that can help with a wide range of tasks and provide
valuable insights and information on a wide range of topics. Whether you need help with a
specific question or just want to have a conversation about a particular topic, Assistant
is here to assist.

TOOLS:
------

Assistant has access to the following tools:

wikipedia_search - searches the wikipedia database for the answer
web_search - searches the web for the answer
calculator - calculates the answer to the question
weather_api - gets the weather for the location


To use a tool, please use the following format:

```
Thought: Do I need to use a tool? Yes
Action: the action to take, should be one of [wikipedia_search, web_search, calculator,
weather_api]
Action Input: the input to the action
Observation: the result of the action
```

When you have a response to say to the Human, or if you do not need to use a tool, you
MUST use the format:

```
Thought: Do I need to use a tool? No
Final Answer: [your response here]
```

Begin!


New input: Whow was King Arthur?
model
## Review of Assistant's Response

This text describes Assistant, a large language model designed to provide assistance with
various tasks. It covers the model's capabilities, tools available, and how to use them.

**Strengths:**

* **Clear and concise:** The text is well-structured and easy to read, with clear sections
and concise sentences.
* **Informative:** It provides a comprehensive overview of Assistant's functionalities and
capabilities.
* **Friendly and helpful:** The tone is friendly and helpful, guiding the user through the
different tools and formats for using the model.
* **Well-organized:** The text clearly outlines tools available, their functions, and how
to utilize them.

**Areas for Improvement:**

* **Lack of specific examples:** While the text mentions the ability to engage in
discussions and provide explanations on various topics, it lacks concrete examples of such
interactions.
* **Missing information:** It does not explain the limitations of Assistant or potential
challenges it may face.
* **Unclear call to action:** The text concludes with an open-ended ending without a clear
call to action for the user.

**Overall:**

This text effectively introduces Assistant and its capabilities, but could benefit from
additional details and examples to further illustrate its functionalities and potential
uses.

CPU times: user 13.2 s, sys: 19.7 ms, total: 13.2 s
Wall time: 13.2 s


In [ ]:

react_prompt = """Assistant is a large language model trained by Google.

Assistant is designed to be able to assist with a wide range of tasks, from answering simple questions to providing in-depth explanations and discussions on a wide range of topics. As a language model, Assistant is able to generate human-like text based on the input it receives, allowing it to engage in natural-sounding conversations and provide responses that are coherent and relevant to the topic at hand.

Assistant is constantly learning and improving, and its capabilities are constantly evolving. It is able to process and understand large amounts of text, and can use this knowledge to provide accurate and informative responses to a wide range of questions. Additionally, Assistant is able to generate its own text based on the input it receives, allowing it to engage in discussions and provide explanations and descriptions on a wide range of topics.

Overall, Assistant is a powerful tool that can help with a wide range of tasks and provide valuable insights and information on a wide range of topics. Whether you need help with a specific question or just want to have a conversation about a particular topic, Assistant is here to assist.

TOOLS:
------

Assistant has access to the following tools:

wikipedia_search - searches the wikipedia database for the answer\n
web_search - searches the web for the answer\n
calculator - calculates the answer to the question\n
weather_api - gets the weather for the location\n


To use a tool, please use the following format:

```
Thought: Do I need to use a tool? Yes
Action: the action to take, should be one of [wikipedia_search, web_search, calculator, weather_api]
Action Input: the input to the action
Observation: the result of the action
```

When you have a response to say to the Human, or if you do not need to use a tool, you MUST use the format:

```
Thought: Do I need to use a tool? No
Final Answer: [your response here]
```

Begin!


New input: What is the latest AI news today?
""" #{agent_scratchpad}

In [ ]:
%%time
generate(react_prompt,
         system_prompt="",
         max_length=512)

user
You are a friendly and helpful assistant

Assistant is a large language model trained by Google.

Assistant is designed to be able to assist with a wide range of tasks, from answering
simple questions to providing in-depth explanations and discussions on a wide range of
topics. As a language model, Assistant is able to generate human-like text based on the
input it receives, allowing it to engage in natural-sounding conversations and provide
responses that are coherent and relevant to the topic at hand.

Assistant is constantly learning and improving, and its capabilities are constantly
evolving. It is able to process and understand large amounts of text, and can use this
knowledge to provide accurate and informative responses to a wide range of questions.
Additionally, Assistant is able to generate its own text based on the input it receives,
allowing it to engage in discussions and provide explanations and descriptions on a wide
range of topics.

Overall, Assistant is a powerful tool that can help with a wide range of tasks and provide
valuable insights and information on a wide range of topics. Whether you need help with a
specific question or just want to have a conversation about a particular topic, Assistant
is here to assist.

TOOLS:
------

Assistant has access to the following tools:

wikipedia_search - searches the wikipedia database for the answer

web_search - searches the web for the answer

calculator - calculates the answer to the question

weather_api - gets the weather for the location



To use a tool, please use the following format:

```
Thought: Do I need to use a tool? Yes
Action: the action to take, should be one of [wikipedia_search, web_search, calculator,
weather_api]
Action Input: the input to the action
Observation: the result of the action
```

When you have a response to say to the Human, or if you do not need to use a tool, you
MUST use the format:

```
Thought: Do I need to use a tool? No
Final Answer: [your response here]
```

Begin!


New input: What is the latest AI news today?
model
## Summary of Assistant's text:

This text describes Assistant, a large language model designed to assist with various
tasks. It highlights its ability to engage in natural-sounding conversations, provide
coherent and relevant responses, and learn and improve continuously. It also lists the
tools available to Assistant, including Wikipedia search, web search, calculator, and
weather API.

**Key takeaways:**

* **Assistant is a powerful tool:** It can handle a wide range of tasks and provide
valuable insights and information.
* **It has access to various tools:** This allows it to perform complex actions like
searching the web, retrieving information from Wikipedia, and calculating calculations.
* **It is constantly learning:** Assistant is constantly learning and improving its
capabilities.
* **It has a clear format for interaction:** There is a standardized format for using
Assistant, making it easy to interact with.

**Overall, Assistant is a versatile and helpful tool for various tasks and information
needs.**

CPU times: user 9.82 s, sys: 10.9 ms, total: 9.83 s
Wall time: 9.81 s
